In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

# Main Data analysis file

## Introduction

This is the main data analysis file which does all the analysis task, answers all questions 
and is so to say the major part of the report.
The major self set task during the data analysis was to make sense of the counted cyclist data and try to evaluate traffic 
not so much as a function of time, but moreover with spatial revolution.
This is somewaht difficult since directional traffic information is not supplied, but one can only make sense of the data by trying to reconstruct the underlying traffic

I used two methods: 1)Visualise traffic on a map and by that gain some insight 2)try to mathematically attack the problem by other means

This is interesting as you would want to know more informatin than just numbers from such big datasets, yet this is not a simple task

### Use of the files

Some of the files are so big that it was nesesairy to generate them after download. Github is a major headake with files bigger than 50Mbyte. Running the code in All_Bikers.ipynb is not necesairy though open_street_ map_ data_aquisition.ipynb must be executed prior to running this file. This can be unfortunately quite tideous 


First load pretreated data

In [ ]:
ALL_Bikers=pd.read_csv('All_bikers.csv')
ALL_Bikers

Transform data again to datetime as it does not stay like that
Second plot how much data is available when from Helsinki Espoo and Vanta
Finally make some indices to have easier access to the respective columns

In [ ]:
ALL_Bikers=pd.read_csv('All_bikers.csv')
ALL_Bikers=ALL_Bikers.reset_index()
ALL_Bikers['Date']=pd.to_datetime(ALL_Bikers['Date'])
ALL_Bikers=ALL_Bikers.set_index('Date')
ALL_Bikers=ALL_Bikers.iloc[:,1:]

ax=ALL_Bikers.plot(y="nData", figsize=[20,10])
ALL_Bikers.plot(ax=ax,y='nData_HEL')
ALL_Bikers.plot(ax=ax,y='nData_ESP')
ALL_Bikers.plot(ax=ax,y='nData_VAN')

plt.show()
cIndex_Vantaa=pd.Index(['Asolanvayla P', 'Asolanvayla E', 'Hakunilantie P',
       'Hakunilantie E', 'Kyytitie I', 'Kyytitie L', 'Kaislaranta P',
       'Kaislaranta E', 'Kuusijarvi P', 'Kuusijarvi E', 'Kytopuisto P',
       'Kytöpuisto E', 'Pellas I', 'Pellas L', 'Simonkylantie P',
       'Simonkylantie E', 'Solkikuja I', 'Solkikuja L', 'Vanha Porvoontie P',
       'Vanha Porvoontie E', 'Vantaanlaaksontie P', 'Vantaanlaaksontie E',
       'Ylastontie I', 'Ylastontie L'],
      dtype='object')
cIndex_Espoo=pd.Index(['Espoon portti (Eco-Counter)', 'Espoonlahdenraitti (Eco-Counter)',
       'Gallen-Kallela (Eco-Counter)', 'Gallen-Kallelan tie (DSL10)',
       'Haukilahti (Viacount)', 'Kalevalantien alikulku (DSL10)',
       'Keha I, Laajalahti (DSL10)', 'Keha I, Laajalahti (Eco-counter)',
       'Keilaniemi, Keilaterassi (Viacount)',
       'Keskuspuisto, etelahaara, ita (Viacount 2)',
       'Keskuspuisto, etelahaara, lansi (Viacount 2)',
       'Keskuspuisto, Grimangen (Viacount 2)',
       'Keskuspuisto, Mossenkar (Viacount 2)', 'Kiltapolku (Viacount 2)',
       'Kiltaraitti, etela (Viacount 2)',
       'Kiltaraitti, pohjoinen (Viacount 2)', 'Kirkkojarventie (DSL10)',
       'Kivenlahden uimaranta (Viacount)', 'Kivenlahti (Viacount)',
       'Lansivayla (Eco-counter)', 'Lansivayla, Karhusaari (DSL10)',
       'Martinsillantie (DSL10)', 'Merituulentie (DSL10)',
       'Olarinkatu (Eco-Counter)', 'Otaniemi, Otaranta (Viacount)',
       'Pitkajarventie (DSL10)', 'Pohjantien ylikulku (DSL10)',
       'Pohjantien ylikulku (Eco-Counter)', 'Pohjantien ylikulku (Viacount 2)',
       'Paivankestamonpolku, Kera (Eco-Counter)',
       'Rantaradanraitti, Helsingin raja (Viacount 2)', 'Soukka (Viacount)',
       'Suomenlahdentie (Eco-Counter)',
       'Suomenlahdentie, etelainen (viacount)',
       'Suomenlahdentie, pohjoinen (Viacount)',
       'Tapiola,Lansituulenkuja (Eco-Counter)',
       'Turuntie, Rantaradanreitti (Eco-Counter)',
       'Vihdintie, Kalajarven itapuoli (Viacount 2)',
       'Vihdintie, Uusmaki (DSL10)', 'Ylismaentie,Suurpelto (Eco-Counter)'],
      dtype='object')
cIndex_Helsinki=pd.Index(['Auroransilta', 'Etelaesplanadi', 'Huopalahti (station)',
       'Kaisaniemi/Elaintarhanlahti', 'Kaivokatu', 'Kulosaari bridge south',
       'Kulosaaren silta po. ', 'Kuusisaarentie', 'Kapyla, Pohjoisbaana',
       'Lauttasaarin bridge south side', 'Merikannontie',
       'Munkkiniemi bridge south side', 'Munkkiniemi bridge north side',
       'Heperia park/Ooppera', 'Pitkasilta itapuoli', 'Pitkasilta west side',
       'Lauttasaari bridge north side', 'Ratapihantie', 'Viikintie', 'Baana'],
      dtype='object')

## Data Set

The data is not complete (see figures above and below) but moreover especially for vantaa and espoo there is a lot of voids in the data represented by Nones. This is tricky since especially the mathematical models cannot deal with such voids.
Notheless grouping can also be successfully applied here as we will see for the first task

Yet for the second task we will probably use the data of mid 2018 - end 2019 where there is least voids

Another very difficult aspect was the incompleteness of the positional data, respective the positions separately provides were in a weird coordinate system which is used inside Finland for mapping. Thus combining positional data with time series data was a bit tideous.

In [ ]:
ALL_Bikers.loc[ALL_Bikers.nData>=30].plot(y='nData',marker='o',linestyle='None')

## Preprocessing
Preprosessing was separately done in 2 extra files. One dealt with the extra treatment of the data supplied by Kata to include own needed properties (ALL_Bikers) and the other one deals with some preprosessing related to gathering data from open streetmap. Used libraries apart from standard and minor ones are Pandas, GeoPandas, osmnx, networkx, scipy, (numpy)
and sklearn for PCA

### Main plotting function plus explanation snapshots for powerpoint

In [ ]:
import osmnx as ox
import networkx as nx
from geopandas import GeoSeries
import matplotlib.tri as tri
import matplotlib.colors as colors
import matplotlib as mpl
from scipy.spatial import Delaunay, delaunay_plot_2d

def plot_bikers(HEL_boundary,ESP_boundary,VAN_boundary,HEL_landuse,ESP_landuse,VAN_landuse,asemat,column,G_simple,title,vmin,vmax,snapshots=False,barlabel=True):
    
    fig, ax = plt.subplots(nrows=1,figsize=[20,10])
    
    ax2 = plt.axes([0.26, 0.6, .24, .255])
    
    ax.set_xlim([360000, 402500])
    ax.set_ylim([6665000, 6697500])
    ax2.set_xlim([377500, 390000])
    ax2.set_ylim([6670000, 6677500])
    ax.set_axis_off()
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax.set_title(title,fontdict={'fontsize':30})
    
    ax=HEL_boundary.plot(ax=ax,edgecolor="red",zorder=4)
    ax=ESP_boundary.plot(ax=ax,edgecolor="red",zorder=5)
    ax=VAN_boundary.plot(ax=ax,edgecolor="red",zorder=6)
    ax2=HEL_boundary.plot(ax=ax2,edgecolor="red",zorder=4)
    ax2=ESP_boundary.plot(ax=ax2,edgecolor="red",zorder=5)
    ax2=VAN_boundary.plot(ax=ax2,edgecolor="red",zorder=6)
    
    ax=HEL_landuse.plot(ax=ax,facecolor="grey",alpha=0.8,zorder=1)
    ax=ESP_landuse.plot(ax=ax,facecolor="grey",alpha=0.8,zorder=2)
    ax=VAN_landuse.plot(ax=ax,facecolor="grey",alpha=0.8,zorder=3)
    ax2=HEL_landuse.plot(ax=ax2,facecolor="grey",alpha=0.8,zorder=1)
    ax2=ESP_landuse.plot(ax=ax2,facecolor="grey",alpha=0.8,zorder=2)
    ax2=VAN_landuse.plot(ax=ax2,facecolor="grey",alpha=0.8,zorder=3)

    if snapshots: # make figures step by step and save to illustrate the procedures in plotting
        figsnap, axsnap = plt.subplots(nrows=1,figsize=[20,10])
        ax2snap = plt.axes([0.26, 0.6, .24, .255])
        axsnap.set_xlim([360000, 402500])
        axsnap.set_ylim([6665000, 6697500])
        ax2snap.set_xlim([377500, 390000])
        ax2snap.set_ylim([6670000, 6677500])
        axsnap.set_axis_off()
        ax2snap.set_xticks([])
        ax2snap.set_yticks([])
        axsnap.set_title(title,fontdict={'fontsize':30})
    
        axsnap=HEL_boundary.plot(ax=axsnap,edgecolor="red",zorder=4)
        axsnap=ESP_boundary.plot(ax=axsnap,edgecolor="red",zorder=5)
        axsnap=VAN_boundary.plot(ax=axsnap,edgecolor="red",zorder=6)
        ax2snap=HEL_boundary.plot(ax=ax2snap,edgecolor="red",zorder=4)
        ax2snap=ESP_boundary.plot(ax=ax2snap,edgecolor="red",zorder=5)
        ax2snap=VAN_boundary.plot(ax=ax2snap,edgecolor="red",zorder=6)
    
        axsnap=HEL_landuse.plot(ax=axsnap,facecolor="grey",alpha=0.8,zorder=1)
        axsnap=ESP_landuse.plot(ax=axsnap,facecolor="grey",alpha=0.8,zorder=2)
        axsnap=VAN_landuse.plot(ax=axsnap,facecolor="grey",alpha=0.8,zorder=3)
        ax2snap=HEL_landuse.plot(ax=ax2snap,facecolor="grey",alpha=0.8,zorder=1)
        ax2snap=ESP_landuse.plot(ax=ax2snap,facecolor="grey",alpha=0.8,zorder=2)
        ax2snap=VAN_landuse.plot(ax=ax2snap,facecolor="grey",alpha=0.8,zorder=3)
        i=1
        plt.gcf().savefig(f'./pictures/snapshot{i:02}.png', bbox_inches='tight', dpi=400);
        # First snapshot cotaining only basic OSM stuff
        
        points=[]
        for point in asemat.geometry:
            point=point.coords[:][0]
            points.append([point[0], point[1]])
        points=np.array(points)
        points=np.unique(points,axis=0)
        tri = Delaunay(points)

        figsnap = delaunay_plot_2d(tri,ax=axsnap)
        fig2snap = delaunay_plot_2d(tri,ax=ax2snap)
        
        i=2
        plt.gcf().savefig(f'./pictures/snapshot{i:02}.png', bbox_inches='tight', dpi=400);
        # Second show the Delaunay used to find paths


    
    G_simple_gdf_nodes,G_simple_gdf_edges=ox.graph_to_gdfs(G_simple)
    G_simple_gdf_edges.plot(ax=ax,linewidth=1,zorder=7,color='red')
    G_simple_gdf_edges.plot(ax=ax2,linewidth=1,zorder=7,color='red')

    ax=asemat.plot(ax=ax,markersize=40,column=column,zorder=8,vmin=vmin, vmax=vmax,norm=colors.Normalize(vmin,vmax))
    ax2=asemat.plot(ax=ax2,markersize=40,column=column,zorder=8,vmin=vmin, vmax=vmax,norm=colors.Normalize(vmin,vmax))

    if snapshots:
        G_simple_gdf_edges.plot(ax=axsnap,linewidth=1,zorder=7,color='red')
        G_simple_gdf_edges.plot(ax=ax2snap,linewidth=1,zorder=7,color='red')

        axsnap=asemat.plot(ax=axsnap,markersize=40,column=column,zorder=8,vmin=vmin, vmax=vmax,norm=colors.Normalize(vmin,vmax))
        ax2snap=asemat.plot(ax=ax2snap,markersize=40,column=column,zorder=8,vmin=vmin, vmax=vmax,norm=colors.Normalize(vmin,vmax))
        i=3
        plt.gcf().savefig(f'./pictures/snapshot{i:02}.png', bbox_inches='tight', dpi=400);
        # Third snapshot shows the paths generated from the triangulation, finding paths with networkx

    
    
    
    z=asemat[column].values
    pos=np.array([points.coords[:][0] for points in asemat.geometry])
    x=pos[:,0]
    y=pos[:,1]
    
    #https://matplotlib.org/3.1.0/gallery/images_contours_and_fields/irregulardatagrid.html#sphx-glr-gallery-images-contours-and-fields-irregulardatagrid-py
    #ax.tricontour(x, y, z, levels=20, linewidths=0.5, colors='k')
    cntr1 = ax.tricontourf(x, y, z, levels=20, cmap="RdBu_r",vmin=vmin, vmax=vmax,norm=colors.Normalize(vmin,vmax))
    cbar=fig.colorbar(mpl.cm.ScalarMappable(norm=colors.Normalize(vmin,vmax),cmap="RdBu_r"),ax=ax)
    cbar.ax.tick_params(axis='y', labelsize=18)
    if barlabel:
        cbar.set_label(label='Bike Counts/h',fontsize=20)
    cntr2 = ax2.tricontourf(x, y, z, levels=20, cmap="RdBu_r",vmin=vmin, vmax=vmax,norm=colors.Normalize(vmin,vmax))
    
    if snapshots:
        cntr1snap = axsnap.tricontourf(x, y, z, levels=20, cmap="RdBu_r",vmin=vmin, vmax=vmax,norm=colors.Normalize(vmin,vmax))
        cbarsnap=figsnap.colorbar(mpl.cm.ScalarMappable(norm=colors.Normalize(vmin,vmax),cmap="RdBu_r"),ax=axsnap)
        cbarsnap.ax.tick_params(axis='y', labelsize=18)
        if barlabel:
            cbarsnap.set_label(label='Bike Counts/h',fontsize=20)
        cntr2snap = ax2snap.tricontourf(x, y, z, levels=20, cmap="RdBu_r",vmin=vmin, vmax=vmax,norm=colors.Normalize(vmin,vmax))
        i=4
        plt.gcf().savefig(f'./pictures/snapshot{i:02}.png', bbox_inches='tight', dpi=400);
        plt.clf();# close figure that was used only for snapshots
        # Last snapshot showing all, including contourplot

    
    return fig,ax,ax2



city1 = ox.gdf_from_place('Helsinki, Finland').geometry.unary_union
city2 = ox.gdf_from_place('Espoo, Finland').geometry.unary_union
city3 = ox.gdf_from_place('Vantaa, Finland').geometry.unary_union
city4 = ox.gdf_from_place('Kauniainen, Finland').geometry.unary_union
HEL_ESP_VAN_KAU=GeoSeries([city1,city2,city3,city4]).unary_union

asemat=gpd.read_file('.\Helsingin_seudun_pyörälaskennat\Helsingin_seudun_pyörälaskennat.shp')

HEL_ESP_VAN_KAU=gpd.GeoDataFrame(pd.DataFrame(data=[HEL_ESP_VAN_KAU],columns=['geometry']))
asemat=asemat.loc[asemat.within(HEL_ESP_VAN_KAU.geometry[0])]
asemat=ox.project_gdf(asemat)

HEL_boundary=gpd.read_file('.\Helsinki\Helsinki.shp').boundary
ESP_boundary=gpd.read_file('.\Espoo\Espoo.shp').boundary
VAN_boundary=gpd.read_file('.\Vantaa\Vantaa.shp').boundary
HEL_landuse=gpd.read_file('.\Helsinki_landuse\Helsinki_landuse.shp')
ESP_landuse=gpd.read_file('.\Espoo_landuse\Espoo_landuse.shp')
VAN_landuse=gpd.read_file('.\Vantaa_landuse\Vantaa_landuse.shp')

G_simple=ox.load_graphml('HEL_ESP_VAN_KAU_bike_routes_simple.graphml',folder='.')

fig,ax,ax2=plot_bikers(HEL_boundary,ESP_boundary,VAN_boundary,HEL_landuse,ESP_landuse,VAN_landuse,asemat,'huip_2018',G_simple,'Maximum bike counts 2018',0,9000,snapshots=True)





Use imagemagick to make animated gif and display it

In [ ]:
!magick convert -delay 2500 -loop 0 ./pictures/snapshot*.png ./pictures/method.gif

from IPython.display import Image;
Image("./pictures/method.gif")

Prepare main data for plotting

In [ ]:
ALL_Bikers_arkki=ALL_Bikers.loc[ALL_Bikers.arkkipaiva].copy()

ALL_Bikers_arkki=ALL_Bikers_arkki.groupby('Time').mean().T.iloc[7:,:]
ALL_Bikers_arkki['x']=0.0
ALL_Bikers_arkki['y']=0.0
ALL_Bikers_arkki['Nimi2']=''
ALL_Bikers_arkki['Kunta']=''
ALL_Bikers_arkki.iloc[0:20,27]='Helsinki'
ALL_Bikers_arkki.loc[cIndex_Vantaa,'Kunta']='Vantaa'
ALL_Bikers_arkki.loc[cIndex_Espoo,'Kunta']='Espoo'
ALL_Bikers_arkki=ALL_Bikers_arkki.reset_index().set_index(['Kunta','index']).sort_values(['Kunta','index'])
ALL_Bikers_arkki.loc[('Espoo',slice(None))]

#asemat.iloc[:,0:4].sort_values(['Kunta','Paikka']).to_excel('asematpos.xls')

This export was used to create a handcrafted position file which is later treated here

prepare for join

In [ ]:
ALL_Bikers_arkki=ALL_Bikers_arkki.reset_index().set_index('Kunta')
ALL_Bikers_arkki.loc['Helsinki','Nimi2']=HEL_pisteet['Unnamed: 0'].values
ALL_Bikers_arkki.loc['Helsinki','x']=HEL_pisteet.x.values
ALL_Bikers_arkki.loc['Helsinki','y']=HEL_pisteet.y.values
ALL_Bikers_arkki.reset_index().set_index(['Kunta','index']).iloc[51:61,24:]
ALL_Bikers_arkki['Nr']=np.arange(84)
ALL_Bikers_arkki

load postion data

In [ ]:
Positions=pd.read_excel('Positions.xls')
Positions


and generate geometries for geodataframe

In [ ]:
from shapely.geometry import Point
geom=[]
x=[]
y=[] 
ID=[]
for n in range(len(Positions)):
    if Positions.loc[n,'x']<100:
        P2=Positions.loc[n,'x']
        point=asemat.loc[asemat.Id==P2,'geometry'].values[0]
        geom.append(point)
        x.append(point.coords[0][0])
        y.append(point.coords[0][0])
        ID.append(P2)
    else:
        geom.append(np.nan)
        x.append(np.nan)
        y.append(np.nan)
        ID.append(np.nan)

Positions['geometry']=geom
Positions['x']=x
Positions['y']=y
Positions['ID']=ID
Positions=Positions.dropna()
Positions

Remove nans and convert to float

In [ ]:
ALL_Bikers_arkki_dna=ALL_Bikers_arkki.copy()

ALL_Bikers_arkki_dna['ID']=ID
ALL_Bikers_arkki_dna['x']=x
ALL_Bikers_arkki_dna['y']=y
ALL_Bikers_arkki_dna['geometry']=geom


ALL_Bikers_arkki_dna=ALL_Bikers_arkki_dna.dropna()

cols = ALL_Bikers_arkki_dna.columns
cols=cols[1:25]
ALL_Bikers_arkki_dna[cols] = ALL_Bikers_arkki_dna[cols].apply(pd.to_numeric, errors='coerce')

ALL_Bikers_arkki_dna.info()

In [ ]:
ALL_Bikers_arkki_dna

Grouping of the main data, using average data

In [ ]:
ALL_Bikers_taulukko=ALL_Bikers_arkki_dna.groupby('ID').sum().drop(columns=['x','y','Nr'])
ALL_Bikers_taulukko

Drop some columns and join then make a geodataframe for plotting

In [ ]:
id_geom=Positions.copy().set_index('ID').drop(columns=['index','Nimi2','x','y','Nr'])
ALL_Bikers_taulukko=ALL_Bikers_taulukko.join(id_geom)
ALL_Bikers_taulukko=gpd.GeoDataFrame(ALL_Bikers_taulukko)
ALL_Bikers_taulukko

Prepare column index for plotting

In [ ]:
columns=ALL_Bikers_taulukko.columns
columns=columns[0:24]
n_col=len(columns)
columns


## Plotting

In [ ]:
for i in range(24):    
    fig,ax,ax2=plot_bikers(HEL_boundary,ESP_boundary,VAN_boundary,HEL_landuse,ESP_landuse,VAN_landuse,ALL_Bikers_taulukko,columns[i],G_simple,f'Weekdays, average counts at {columns[i]}',0,500)
    plt.gcf().savefig(f'./pictures/AVG_traff{i:02}.png', bbox_inches='tight', dpi=400);
    plt.clf();


Use again imagemagick to make an animated gif

In [ ]:
#!magick convert -delay 500 -loop 0 ./pictures/AVG_traff*.png ./pictures/AVG_traff.gif

from IPython.display import Image;
Image("./pictures/AVG_traff.gif")

Select data from 2018-2019 for PCA

In [ ]:
ALL_Bikers_useful=ALL_Bikers.copy().loc['2018-07-01':,:]
ALL_Bikers_useful=ALL_Bikers_useful.loc[ALL_Bikers_useful.nData>=30,:]

In [ ]:
ALL_Bikers_useful.plot(y='nData',marker='o',linestyle='None')

In [ ]:
ALL_Bikers_useful=ALL_Bikers_useful.loc[ALL_Bikers_useful.arkkipaiva].T.dropna().T
ALL_Bikers_select=ALL_Bikers_useful.copy().loc[(ALL_Bikers_useful.Time=='07:00:00')|(ALL_Bikers_useful.Time=='07:00:00')]

ALL_Bikers_select.iloc[:,10:]

In [ ]:
cols=ALL_Bikers_select.columns
cols=cols[10:]

for n in range(len(cols)):
    ALL_Bikers_select.loc[:,cols[n]] = ALL_Bikers_select.loc[:,cols[n]].apply(pd.to_numeric)

ALL_Bikers_select.info()




Now group , calculate averages to remove nans and join Data with positions

In [ ]:
Data=ALL_Bikers_select.groupby(['month','Weekday']).mean().T
Data_HEL=Data.iloc[0:17].sort_index()
Data_ESP=Data.iloc[17:].sort_index()

ESP=[0,None,6,20,23,25,32,36,None]
HEL=[40,41,42,43,44,45,46,47,48,49,50,52,53,54,56,57,58]
Data_HEL['Nr']=HEL
Data_ESP['Nr']=ESP

#Data_HEL.merge(Data_ESP)
Data=pd.concat([Data_ESP,Data_HEL],axis=0).dropna()
Data=Data.reset_index().set_index('Nr')
Pos=Positions.set_index('Nr')

index=Data.index
geom=np.array(Pos.loc[index].geometry)
Data['geometry']=geom
Data=Data.set_index('index')
Data


How do the correlations between the counts look like?

In [ ]:
corr_table=pd.DataFrame(Data.iloc[:,1:-1].values.astype('Float64').T).corr()


#corr_table.columns=Mon_8_9_1.columns[9:]
#corr_table.index=Mon_8_9_1.columns[9:]
corr_table

plt.pcolor(corr_table)
corr_table

Now calculate the first n principal components

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

#print(ALL_arkki_8.iloc[:,9:])

# Try first all components
X=Data.iloc[:,1:-1].values.astype('Float64')
pca = PCA()
pca.fit(X)
print('Singular values :\n {}\n'.format(pca.singular_values_))
print('Explained variance /%  :\n {}\n'.format(pca.explained_variance_ratio_*100))
print('Principal components :\n {} \n'.format( pca.components_))

P=pca.components_[:,0:3]




In [ ]:
Dataplot=gpd.GeoDataFrame(geometry=Data.iloc[:,-1].values)
Dataplot['P0']=P[:,0]
Dataplot['P1']=P[:,1]
Dataplot['P2']=P[:,2]
Dataplot

And plot them into a gif animation

In [ ]:
columns=Dataplot.columns
columns=columns[1:4]
n_col=len(columns)
columns

In [ ]:
for i in range(3):    
    fig,ax,ax2=plot_bikers(HEL_boundary,ESP_boundary,VAN_boundary,HEL_landuse,ESP_landuse,VAN_landuse,Dataplot,columns[i],G_simple,f'Principal component Nr.  {i+1}',-0.5,0.5,barlabel=False)
    plt.gcf().savefig(f'./pictures/PC{i+1:01}.png', bbox_inches='tight', dpi=400);
    plt.clf();

In [ ]:
!magick convert -delay 500 -loop 0 ./pictures/PC*.png ./pictures/PCA.gif

from IPython.display import Image;
Image("./pictures/PCA.gif")

## Conclusion
It appears that visualising the data normally gives a better insight into the traffic of bikes, yet it is very difficutlt to find out from where to where. Also, correlating the count data does neither give any particular patterns in the covariane matrix, nor does the PCA result in any more meaningful patterns that could help to clarify the  direction of traffic.

Another possibility is that the data quality is not good enough to make such an analysis possible.

Thus, in this case at least, visualising the data and inspection by eye gives more insight than a pure mathematical approach.